In [ ]:
import cv2 as cv
import numpy as np

# [1] 모델 이미지(버스) 일부를 잘라서 사용하고, 그레이스케일로 변환
img1 = cv.imread('mot_color70.jpg')[190:350, 440:560]  # ROI 크롭 (버스 부분)
gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)

# [2] 장면 이미지(전체), 그레이스케일로 변환
img2 = cv.imread('mot_color83.jpg')
gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

# [3] SIFT 객체 생성 → 두 이미지에서 KeyPoint + Descriptor 추출
sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)  # 모델
kp2, des2 = sift.detectAndCompute(gray2, None)  # 장면

# [4] FLANN 매칭기 생성 → KNN 매칭 (각 특징점당 최근접 2개)
flann_matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match = flann_matcher.knnMatch(des1, des2, 2)

# [5] Ratio Test로 좋은 매칭만 걸러냄
T = 0.7  # 임계값
good_match = []
for nearest1, nearest2 in knn_match:
    if (nearest1.distance / nearest2.distance) < T:
        good_match.append(nearest1)

# [6] 좋은 매칭 쌍의 좌표를 가져오기
#  → points1 = 모델 이미지 좌표
#  → points2 = 장면 이미지 좌표
points1 = np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2 = np.float32([kp2[gm.trainIdx].pt for gm in good_match])

# [7] Homography 변환 행렬 추정 (RANSAC으로 이상치 제거)
H, _ = cv.findHomography(points1, points2, cv.RANSAC)

# [8] 모델 이미지의 경계 박스 4점 정의
h1, w1 = img1.shape[0], img1.shape[1]
h2, w2 = img2.shape[0], img2.shape[1]
box1 = np.float32([[0, 0], [0, h1 - 1], [w1 - 1, h1 - 1], [w1 - 1, 0]]).reshape(4, 1, 2)

# [9] Homography로 모델 경계 박스를 장면 이미지로 투영
box2 = cv.perspectiveTransform(box1, H)

# [10] 장면 이미지 위에 초록색 경계 박스 그리기
img2 = cv.polylines(img2, [np.int32(box2)], True, (0, 255, 0), 8)

# [11] 매칭 결과를 하나의 큰 이미지에 시각화 (점과 연결선)
img_match = np.empty((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
cv.drawMatches(
    img1, kp1,        # 모델 이미지 + KeyPoint
    img2, kp2,        # 장면 이미지 + KeyPoint
    good_match,       # 좋은 매칭 쌍
    img_match,        # 결과 저장 캔버스
    flags=cv.DrawMatchesFlags.NOT_DRAW_SINGLE_POINTS
)

# [12] 결과 출력
cv.imshow('Matches and Homography', img_match)
cv.waitKey()
cv.destroyAllWindows()


| 파트            | 역할                                 |
| ------------- | ---------------------------------- |
| `SIFT`        | 모델 & 장면에서 KeyPoint + Descriptor 추출 |
| `FLANN + KNN` | 디스크립터 매칭, 최근접 2개                   |
| `Ratio Test`  | 모호한 매칭 제거                          |
| `Homography`  | 점쌍들로 평면 투영 관계 추정                   |
| `polylines`   | 모델 경계 상자를 장면에 표시                   |
| `drawMatches` | 매칭 쌍 시각화                           |